In [85]:
import statisUtils as utils
import pandas as pd
import numpy as np
from scipy.special import gamma, binom

In [2]:
df = pd.read_csv('data/MEF_QC_all.csv').transpose()
df.head()

,0,1,2,3,4,5,6,7,8,9,...,2151,2152,2153,2154,2155,2156,2157,2158,2159,2160
0,10,17,1,0,1,5,-1,0,3,7,...,2,0,4,0,1,0,10,1,6,0
2,4,18,0,0,8,1,-1,0,4,8,...,8,0,1,3,0,9,-1,2,24,12
2.1,-1,9,4,4,2,0,-1,0,10,11,...,4,5,5,3,4,3,3,4,21,2
2.2,-1,14,9,0,5,9,0,2,6,5,...,6,1,4,2,-1,4,-1,7,17,6
1,0,0,0,2,10,5,10,0,3,0,...,1,6,4,2,5,0,0,2,20,5


In [7]:
data = np.array(df)
data

array([[10, 17,  1, ...,  1,  6,  0],
       [ 4, 18,  0, ...,  2, 24, 12],
       [-1,  9,  4, ...,  4, 21,  2],
       ...,
       [ 0, 22,  3, ...,  0, 13,  2],
       [ 0,  2,  0, ...,  4,  7,  2],
       [ 6, 12,  3, ...,  9,  1,  0]], dtype=int64)

In [77]:
ron = 5.0
kon = 2
koff=3
roff=6
mu=4
delta=0.5
k=4
Laplace_s = np.array(range(0,k+1))

In [71]:
Lfon = (ron/(Laplace_s + ron))**kon
Lfoff = (roff/(Laplace_s + roff))**koff

In [72]:
mean_tau_off = kon/ron
mean_tau_on = koff/roff

In [73]:
c = 1/(mean_tau_off + mean_tau_on)

In [74]:
LFon = np.insert((1-Lfon[1:])/Laplace_s[1:], 0, mean_tau_off)
LFoff = np.insert((1-Lfoff[1:])/Laplace_s[1:], 0, mean_tau_on)

In [75]:
Ck = np.zeros(k)
Ck[0] = (1 - Lfoff[1])/(mean_tau_off + mean_tau_on)

numpy.float64

In [138]:
for iter in range(2,k+1):
    i = np.array(range(1, iter+1))
    Ck_coef = np.array(
        (mu**i[0:-1])
        * Lfon[iter-i[0:-1]]
        * Ck[iter-1-i[0:-1]]
        / ((1-Lfoff[iter-i[0:-1]]*Lfon[iter-i[0:-1]])*gamma(i[0:-1]+1)))
    Ck_coef_last_element = c*mu**(iter-1)/gamma(iter+1)
    Ck_coef=np.append(Ck_coef,Ck_coef_last_element)
    
    Ck_sum = np.zeros(iter)
    for iter_i in range(1, iter+1):
        for iter_j in range(0, iter_i+1):
            Ck_sum[iter_i-1] = Ck_sum[iter_i-1] + binom(iter_i,iter_j) * (-1)**(iter_i-iter_j)*Lfoff[iter-iter_j]
    Ck[iter-1] = np.sum(Ck_coef*Ck_sum)
    
print(Ck)    

[0.41140266 0.78304908 0.82733613 0.62995267]


In [141]:
bk = np.zeros(k);
for iter in range(1,k+1):
    i = np.array(range(1, iter+1))
    bk_coef_first_element = c*mu**(iter)/gamma(iter+1)
    bk_coef = np.array(
        (mu / iter)
        * (mu**(iter-i[0:-1]))
        * Lfon[i[0:-1]] 
        * Ck[i[0:-1]-1]
        / (gamma(iter-i[0:-1])*(1-Lfon[i[0:-1]]*Lfoff[i[0:-1]]))
    )
    bk_coef = np.insert(bk_coef, 0, bk_coef_first_element)
    bk_sum = np.zeros(iter)
    for iter_i in range(0,iter):
        for iter_j in range(0, iter - iter_i):
            bk_sum[iter_i] = bk_sum[iter_i] + binom(iter-iter_i-1,iter_j) * (-1)**(iter-iter_i-iter_j-1) * LFoff[iter-iter_j-1]
    bk[iter-1] = np.sum(bk_coef*bk_sum)    
print(bk)   

[2.22222222 2.65719951 2.23965393 1.47945764]


In [154]:
b=bk
m = b[0]
v = 2*b[1]+b[0]-b[0]**2
cv2 = v/(m**2)
fano = v/m
sk = (6*b[2]+6*b[1]+b[0]-3*b[0]*(2*b[1]+b[0])+2*b[0]**3)/(2*b[1]+b[0]-b[0]**2)**1.5 + 1
kt = (24*b[3]+36*b[2]+14*b[1]+b[0]-4*b[0]*(6*b[2]+6*b[1]+b[0])
      + 6*b[0]**2*(2*b[1]+b[0])-3*b[0]**4)/(2*b[1]+b[0]-b[0]**2)**2
bc = ((sk - 1)**2 + 1)/kt
bc
statis = [m,cv2,fano,sk,kt,bc]
statis

[2.2222222222222223,
 0.5261658031088082,
 1.169257340241796,
 1.7895827587038013,
 3.6713240553647024,
 0.44219494339380394]